# Music Store Analysis

In [1]:
import mysql.connector

# Connecting jupyter notebook to database

In [2]:
data_base = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "123456",
    database = "music_store"
)
mycursor = data_base.cursor()

## 1. Who is the senior most employee based on job title?

In [3]:
mycursor.execute("SELECT CONCAT(first_name,' ',last_name) FROM music_store.employee ORDER BY levels DESC LIMIT 1")

In [4]:
for x in mycursor:
    print(x)

('Andrew Adams',)


## 2. Which countries have the most Invoices?

In [5]:
mycursor.execute("""SELECT billing_country, COUNT(*) AS total_count FROM music_store.invoice
                    GROUP BY billing_country
                    ORDER BY total_count DESC 
                    LIMIT 1""")

In [6]:
for x in mycursor:
    print(x)

('USA', 131)


## 3. What are top 3 values of total invoice?

In [7]:
mycursor.execute("""SELECT * FROM music_store.invoice
                    ORDER BY total DESC
                    LIMIT 3""")

In [8]:
for x in mycursor:
    print(x)

(183, 42, '2018-02-09 00:00:00', '9, Place Louis Barthou', 'Bordeaux', 'None', 'France', '33000', 23.759999999999998)
(92, 32, '2017-07-02 00:00:00', '696 Osborne Street', 'Winnipeg', 'MB', 'Canada', 'R3L 2B9', 19.8)
(526, 5, '2020-06-08 00:00:00', 'Klanova 9/506', 'Prague', 'None', 'Czech Republic', '14700', 19.8)


## 4. Which city has the best customers? We would like to throw a promotional Music Festival in the city we made the most money. Write a query that returns one city that has the highest sum of invoice totals. Return both the city name & sum of all invoice totals

In [9]:
mycursor.execute("""SELECT billing_country, SUM(total) AS total_invoice FROM music_store.invoice
                    GROUP BY billing_country
                    ORDER BY total_invoice DESC 
                    LIMIT 1""")

In [10]:
for x in mycursor:
    print(x)

('USA', 1040.4899999999998)


## 5. Who is the best customer? The customer who has spent the most money will be declared the best customer. Write a query that returns the person who has spent the most money

In [11]:
mycursor.execute("""SELECT customer.customer_id,first_name , last_name, SUM(invoice.total) AS total_spending FROM music_store.customer
                    JOIN music_store.invoice 
                    ON customer.customer_id = invoice.customer_id
                    GROUP BY customer.customer_id, first_name , last_name
                    ORDER BY total_spending DESC
                    LIMIT 1""")

In [12]:
for x in mycursor:
    print(x)

(5, 'FrantiÅ¡ek', 'WichterlovÃ¡', 144.54000000000002)
